In [1]:
import pandas as pd
import seaborn as sns

In [3]:
# read in data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
file_path = 'https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/B2P%20Rwanda%20Site%20Assessment%20Data_2020.06.03.csv'
df = pd.read_csv(file_path, encoding='latin-1')
print(f'Dimensions of dataset: {df.shape}')
df.head()

In [5]:
# split the Form: Form Name to get the data from the column and convert date to datetime

df['Assessment Date'] = df['Form: Form Name'].str.replace('Project Assessment - ', "")

df['Assessment Date'] = pd.to_datetime(df['Assessment Date'])
print(f'Dimensions of dataset: {df.shape}')
df.head()

In [14]:
# what are the duplicate project codes?
# it looks like that all the duplicates have two values. One has three.
# a quick and simple way to remove the duplicates would keep the sorting by project code and assesment date
# then drop the duplicates and keep the first occurence. That way we have the most recent date. 

# this was our test to find all the duplicates that we needed to drop
# testing = df[df.duplicated(subset='Project Code', keep=False)].sort_values(by=['Project Code', 'Assessment Date'], ascending=False)
# testing.head()
# testing[testing['Individuals Directly Served'].notna()]

# Indices that we 

In [6]:
# manually went thru above duplicates and selected those that needed to be DROPPED. 

indexes_to_drop = [192,210,216,767,202,1119,908,735,901,272,643,756,
                   738,193,655,642,327,909,1274,1289,135,176,1213,
                   493,178,205,387,397,395,398,399,401,372,371,892,
                   839,776,171,169,170,1117,1096,771,1109,95,89,132,133,134,
                   ]


df = df.drop(index=indexes_to_drop)
print(f'Dimensions of dataset: {df.shape}')

Dimensions of dataset: (1423, 28)


In [15]:
# make sure there are no duplicates 

df.duplicated(subset='Project Code', keep=False).value_counts()

False    1423
dtype: int64

In [94]:
# there are some columns that are all null values, we can drop those.
df.isnull().sum()


In [7]:
# these are columns we can drop
col_drop = ['Community Served 6', 'Community Served 7',
            'Community Served 8', 'Community Served 9',
            'Community Served 10'
           ]

# these are columns that we want as Ids when we melt the table
ID_variable = ['Country','Province', 'District', 'Sector', 'Cell', 
               'Bridge Site Name', 'Project Stage', 
               'Project Sub-Stage', 'Project Code', 'Bridge Type',
               ' Span (m)', ' GPS (Latitude)', 'GPS (Longitude)',
               'Individuals Directly Served', 'Form: Form Name',
               'CaseSafeID Form', 'Bridge Opportunity: Opportunity ID',
               'Assessment Date'
              ]

# these are the columns that we want to melt to values
value_variables = ['Community Served 1', 'Community Served 2',
                   'Community Served 3', 'Community Served 4',
                   'Community Served 5'
                  ]

In [9]:
# melt the dataframe so we get all the communities that a bridge would serve in one column
# this will make duplicate observations of bridges. THis is okay since we want to be able to get counts
# of how many villages a bridge would service.

b2p_df = pd.melt(df, id_vars=ID_variable, value_vars=value_variables, var_name='Original_Community_col', value_name='Community_Served', )
print(f'Dimensions of dataset: {b2p_df.shape}')
b2p_df.head()

In [11]:
# now that we have our melted dataframe, we want to delete any observation that has a 
# null value in the 'Community_Served' column

b2p_df = b2p_df[b2p_df['Community_Served'].notna()]
print(f'Dimensions of dataset: {b2p_df.shape}')
b2p_df.head()

In [98]:
# lets check to see if there are still null values that we care about in the dataset

b2p_df.isnull().sum()


In [100]:
b2p_df.sort_values(by='Project Code').head()

In [13]:
# Read in the Data of Government Entities with Government IDs

gov_file_path = 'https://raw.githubusercontent.com/tmbern/Labs25-Bridges_to_Prosperity-TeamC-ds/main/Rwanda%20Administrative%20Levels%20and%20Codes_Province%20through%20Village_2019.02.28.csv'
gov_df = pd.read_csv(gov_file_path, encoding='latin-1')
print(f'Dimensions of dataset: {gov_df.shape}')
gov_df.head()


In [170]:
# District ID dict 
district_ID_dict = gov_df['Dist_ID'].groupby(gov_df['District']).unique().apply(pd.Series).to_dict()[0]
district_ID_dict

In [104]:
# There are the same amount of government Districts as there is in B2P districts. 
len(b2p_df['District'].unique()) == len(gov_df['District'].unique())
# check length of "district" strings on both dfs
sorted(b2p_df['District'].unique()) == sorted(gov_df['District'].unique())

True

In [105]:
# Sector ID 
# check that df['sector'] == gov_df['sector']
len(b2p_df['Sector'].unique()) == len(gov_df['Sector'].unique())

False

In [15]:
# check df['sector'] not in gov_df['sector']
x = [x for x in set(b2p_df['Sector']) if x not in set(gov_df['Sector'])]
len(x)

348

In [16]:
# Strip sectors down to single sector names 
# with regular expressions 
import re

b2p_df['new_sector'] = [re.split("-|~|_|(sector)|(Sector)|(cell)|\(|~", str(i))[0] for i in b2p_df['Sector']]
b2p_df['new_sector'] = b2p_df['new_sector'].str.replace(" ","")
b2p_df['new_sector'].head()

0     Giheke
1     Giheke
2     Buyoga
4    Kayenzi
5    Kayenzi
Name: new_sector, dtype: object

In [108]:
# confirm that df sector list is less than gov list of sectors - important 
b2p_df['new_sector'].nunique() < gov_df['Sector'].nunique()

True

In [61]:
# we group df by province district and sector. 
# get the count of the number of timex that a particular village is in that grouping. 

gov_df.head()

In [17]:
b2p_df['test_cell'] = b2p_df['Cell'].astype(str)

In [81]:
# test_list = [re.split("-|~|_|/|\(|\)", i) for i in b2p_df['test_cell']]

In [18]:
# helper function to clean characters in Cell string
def replace_char(x):
    """
    args: Cell
    returns: Cell value where chr replaced with whitespace
    """
    if type(x) == str:
         char = ['-', '~', '(', ')', '/', '_', 'between', 'villages', 
                'village', ' both', 'Both',' in ' , ' are ', ' village', 
                'all are in', 'village ', ' Village', 'Village ', '&', 'Between', ' Between ', 'Between ', 
                ' and ', ' of ', 'Village', 'Sector', 'sector']
         for i in char:
             if i in x:
                 x = x.replace(i, ' ')
    return x

In [19]:
# apply helper function to Cell column
b2p_df['test_cell'] = b2p_df['Cell'].apply(replace_char)

In [21]:
# Create list of Cells in govt data 
list_of_gov_cells = list(gov_df['Cell'].unique())
sorted(list_of_gov_cells)[600:]

In [242]:
# b2p_df[b2p_df['test_cell'].str.contains('cell',na=False)]

In [22]:
# Fill NaN with "Unknown"
b2p_df['test_cell'] = b2p_df['test_cell'].fillna("Unknown")

In [23]:
# Helper function to
# Complete cleaning of Cell
# return "Unknown for values not in govt data"
def cell_cleaner(x):
    """
    args: Cell
    return: clean value for Cell or "Unknown"
    """
    if type(x) == str:
        for i in list_of_gov_cells:
            if i in x.title():
                return i
            elif "Congo" in x:
                return "Congo-nil"
    return "Unknown"

In [24]:
# Apply helper function 
b2p_df['test_cell'] = b2p_df['test_cell'].apply(cell_cleaner)
b2p_df['test_cell'].nunique()

543

In [25]:
# confirm if helper function worked
x = [x for x in b2p_df['test_cell'] if (x not in list_of_gov_cells) and (x != "Unknown")]
len(x)

0

In [26]:
# Province dictionary. Key is B2P Province name, Value is government province ID
province_dict = {'Northern Province': 4,
                 'Southern Province': 2,
                 'Eastern Province': 5,
                 'Western Province': 3,
                 'Kigali': 1
                 }
# Map Province colummn to Province code
b2p_df['Prov_ID'] = b2p_df.Province.map(province_dict)

In [27]:
# # District ID dict 
district_ID_dict = gov_df['Dist_ID'].groupby(gov_df['District']).unique().apply(pd.Series).to_dict()[0]
# sorted(district_ID_dict)
# map District ID to District 
b2p_df['District_ID'] = b2p_df.District.map(district_ID_dict)

In [28]:
# Change gov_df.Sector to title case 
gov_df['Sector'] = gov_df['Sector'].apply(lambda x: x.title())

In [29]:
gov_sectors = list(gov_df['Sector'].unique())

In [30]:
# fill unknown sectors 
b2p_df['new_sector'] = b2p_df['new_sector'].fillna("Unknown")
b2p_df['new_sector'].isnull().sum()

0

In [31]:
# change new_sector to title case
b2p_df['new_sector'] = b2p_df['new_sector'].apply(lambda x: x.title())

In [32]:
def replace_sectors(x):
    for i in gov_sectors:
        if i in x:
            return i
    return x

In [33]:
b2p_df['new_sector'] = b2p_df['new_sector'].apply(replace_sectors)

In [34]:
corrected_dict = {
                    "Betweenmuhororo": "Muhororo", "Rwanamiro": "Rwaniro", "Rukuzo": "Rukozo", "Rusheshe": "Rusasa",
                    "Gashali": "Gashari", "Bwisìge": "Bwisige", "Gasakka": "Gasaka", "Giko": "Gikondo", "Kabagari": "Kabagali",
                    "Kabengera": "Karengera", "Katabagema": "Katabagemu", "Kibingo": "Kibungo", "Koza": "Kazo", "Mushikir": "Mushikiri",
                    "Mwili": "Mwiri", "Mwumba": "Mamba", "Nkaka": "Nkanka", "Nyagihinga": "Nyagihanga", "Nyakariro": "Nyakaliro",
                    "Rerenge": "Rurenge", "Rugalika": "Rugarika", "Rugamba": "Ruramba", "Buruhukioro": "Buruhukiro",
                    "Bushonyi": "Mushonyi", "Cyingwa": "Cyungo", "Gikheke": "Gikonko", "Mulinga": "Muringa",
                    "Nan": "Unknown", "Nyirangarama": "Ngarama", "Bambiro": "Nyange", "Cyimpindu": "Muhororo","Mutongo": "Muhororo",
                    "Rubirizi": "Kanombe", "Buheta": "Gatebe", "Gushali": "Nyagatare",
}

# replace wrong sectors with correct sector 
b2p_df['new_sector'].replace(corrected_dict,inplace=True)

In [35]:
# find incorrect values 
g = [x for x in b2p_df['new_sector'] if x not in gov_sectors]
len(g)

18

In [36]:
# Unknow sectors - resolve this 
sorted(set(g))

['Gahurizo', 'Kabukuba', 'Kamuragi', 'Nyakabingo', 'Pera', 'Unknown']

In [386]:
b2p_df[b2p_df['new_sector']=='Pera']

,Country,Province,District,Sector,Cell,Bridge Site Name,Project Stage,Project Sub-Stage,Project Code,Bridge Type,Span (m),GPS (Latitude),GPS (Longitude),Individuals Directly Served,Form: Form Name,CaseSafeID Form,Bridge Opportunity: Opportunity ID,Assessment Date,Original_Community_col,Community_Served,new_sector,test_cell,Prov_ID,District_ID
63,Rwanda,Western Province,Rusizi,Pera,NaN,Rusayo,Rejected,Technical,1007661,Suspension,25.0,-2.697308,29.029517,17270.0,Project Assessment - 2018.12.13,a1if1000002qz1JAAQ,006f100000a86ID,2018-12-13,Community Served 1,Pera cell,Pera,Unknown,3,36
1486,Rwanda,Western Province,Rusizi,Pera,NaN,Rusayo,Rejected,Technical,1007661,Suspension,25.0,-2.697308,29.029517,17270.0,Project Assessment - 2018.12.13,a1if1000002qz1JAAQ,006f100000a86ID,2018-12-13,Community Served 2,Kizura cell,Pera,Unknown,3,36


In [85]:
final['Cell'] = final['test_cell']
final['Sector'] = final['new_sector']
final.drop(['Bridge Opportunity: Opportunity ID','Project Sub-Stage','Country',
            ' Span (m)','CaseSafeID Form','test_cell','new_sector'],1,inplace=True)

In [76]:
final.fillna("Unknown",inplace=True)

In [89]:
final[final.isnull().any(axis=1)]

In [99]:
final[final['Cell'] == "Unknown"].shape

(252, 15)

In [111]:
# generate dict of sector and cell IDs from govt data
Sector_ID_dict = gov_df['Sect_ID'].groupby(gov_df['Sector']).unique().apply(pd.Series).to_dict()[0]
Cell_ID_dict = gov_df['Cell_ID'].groupby(gov_df['Cell']).unique().apply(pd.Series).to_dict()[0]


In [139]:
# Insert values for "unknown" sectors and cells 
Cell_ID_dict["Unknown"]  = int(000000)
Sector_ID_dict["Unknown"] = int(000)


In [141]:
# map sector id and cell id to columns
final['Sector_ID'] = final.Sector.map(Sector_ID_dict)
final['Cell_ID'] = final.Cell.map(Cell_ID_dict)


In [146]:
# Covert Sector ID and Cell ID to int
final['Sector_ID'] = final['Sector_ID'].astype(int)
final['Cell_ID'] = final['Cell_ID'].astype(int)

In [161]:
print(final['Sector_ID'].isnull().sum())
print(final['Cell_ID'].isnull().sum())

0

In [148]:
final[final.isnull().any(axis=1)]

In [167]:
final['Sector_ID'] = final['Sector_ID'].fillna(0000)

# cast dtypes as int 
import numpy as np
final['Sector_ID'] = final['Sector_ID'].astype(np.int64)
final['Cell_ID'] = final['Cell_ID'].astype(np.int64)



In [171]:
# confirm final dtypes 
final.dtypes

In [169]:
final.head()

,Province,District,Bridge Site Name,Project Stage,Project Code,Bridge Type,GPS (Latitude),GPS (Longitude),Individuals Directly Served,Assessment Date,Community_Served,Prov_ID,District_ID,Cell,Sector,Sector_ID,Cell_ID
0,Western Province,Rusizi,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Buzi,3,36,Gako,Giheke,3605,370501
1,Western Province,Rusizi,Kamigisha,Rejected,1014106,Suspended,-2.42486,28.9573,Unknown,2018-10-29,Kabuga,3,36,Gako,Giheke,3605,370501
2,Northern Province,Rulindo,Gipfundo,Rejected,1007651,Suspended,-1.72053,30.0812,Unknown,2018-08-11,Gapfundo,4,41,Gahororo,Buyoga,4104,240303
4,Southern Province,Kamonyi,Gisizi,Identified,1014318,Suspended,-1.87087,29.8777,Unknown,2018-11-15,Gisizi,2,28,Kirwa,Kayenzi,2803,320701
5,Southern Province,Kamonyi,Ruheka,Rejected,1014319,Suspended,-1.88396,29.8505,Unknown,2018-11-15,Ruheka,2,28,Kirwa,Kayenzi,2803,320701
